In [ ]:
import re
from abc import ABC, abstractmethod

Part = dict[str, int]

with open("19/input.txt") as f:
    input = f.read()

def parse_input(input: str):
    workflow_str, parts_str = input.split("\n\n")
    return parse_workflows(workflow_str), parse_parts(parts_str)

def parse_parts(parts_str: str):
    parts_lines = parts_str.strip().splitlines()
    parts = []
    for line in parts_lines:
        part = {}
        for property in line[1:-1].split(","):
            name, value = property.split("=")
            value = int(value)
            part[name] = value
        parts.append(part)
    return parts


class Rule:
    def __init__(self, outcome: str) -> None:
        self.outcome = outcome

    def apply(self, part: Part) -> str | None:
        return self.outcome if self._condition(part) else None
    
    def _condition(self, part: Part) -> bool:
        return True
    
    def __repr__(self) -> str:
        return f":{self.outcome}"
    

class ConditionRule(ABC, Rule):
    def __init__(self, outcome: str, property: str, value: int) -> None:
        super().__init__(outcome)
        self.property = property
        self.value = value
    
    @abstractmethod
    def _condition(self, part: Part) -> bool:
        raise NotImplementedError()

class LessThanRule(ConditionRule):
    def _condition(self, part: Part) -> bool:
        part_value = part[self.property]
        return part_value < self.value

    def __repr__(self) -> str:
        return f"{self.property}<{self.value}:{self.outcome}"

class GreaterThanRule(ConditionRule):
    def _condition(self, part: Part) -> bool:
        part_value = part[self.property]
        return part_value > self.value

    def __repr__(self) -> str:
        return f"{self.property}>{self.value}:{self.outcome}"
    

def parse_workflows(workflows_str: str):
    workflows = {}
    workflow_lines = workflows_str.strip().splitlines()
    for line in workflow_lines:
        workflow_name, rules_part = re.match(r"(.*)\{(.*)\}", line).groups()
        rules_defs = rules_part.split(",")
        rules = []

        for rule_def in rules_defs:
            parts = re.match(r"(.*)(>|<)(\d+):(.*)", rule_def)
            if parts is None:
                rules.append(Rule(rule_def))
            else:
                match parts.groups():
                    case prop, "<", value, outcome:
                        rules.append(LessThanRule(outcome, prop, int(value)))
                    
                    case prop, ">", value, outcome:
                        rules.append(GreaterThanRule(outcome, prop, int(value)))

        workflows[workflow_name] = rules
    return workflows

In [ ]:
workflows, parts = parse_input(input)

In [ ]:
def process_part(part: Part, workflows: dict[str, list[Rule]]) -> Part | None:
    workflow = workflows["in"]

    finished = False
    while not finished:
        for rule in workflow:
            outcome = rule.apply(part)
            match outcome:
                case "A":
                    return part
                case "R":
                    return None
                case new_workflow_name if new_workflow_name is not None:
                    workflow = workflows[new_workflow_name]
                    break

In [ ]:
accepted = [
    process_part(p, workflows)
    for p in parts
]
accepted = [p for p in accepted if p is not None]
accepted

In [ ]:
def summarize(accepted: list[Part]):
    return sum(
        p["x"] + p["m"] + p["a"] + p["s"]
        for p in accepted
    )

In [ ]:
workflows

In [ ]:
starting_ranges: dict[str, tuple[int, int]] = {
    p: (1, 4000)
    for p in "xmas"
}
starting_ranges

In [ ]:
from copy import deepcopy

def process_ranges(starting_ranges: dict[str, tuple[int, int]], workflows: dict[str, list[Rule]]):

    processed = []

    to_process = [
        (starting_ranges, "in")
    ]

    while len(to_process) > 0:
        (ranges, workflow_name), *to_process = to_process
        workflow = workflows[workflow_name]


        range_up_to_now = deepcopy(ranges)
        for rule in workflow:
            match rule:
                case LessThanRule(property=property, value=value, outcome=outcome):
                    current_range_start, current_range_end = range_up_to_now[property]

                    if current_range_start < value:
                        new_range_end = min(current_range_end, value-1)
                        new_ranges = deepcopy(range_up_to_now)
                        new_ranges[property] = (current_range_start, new_range_end)

                        if outcome == "A":
                            processed.append(new_ranges)
                        elif outcome != "R":
                            to_process.append((new_ranges, outcome))
                    
                    if current_range_end >= value:
                        new_range_start = max(current_range_start, value)
                        range_up_to_now[property] = (new_range_start, current_range_end)
                    else:
                        break
                
                case GreaterThanRule(property=property, value=value, outcome=outcome):
                    current_range_start, current_range_end = range_up_to_now[property]

                    if current_range_end > value:
                        new_range_start = max(current_range_start, value+1)
                        new_ranges = deepcopy(range_up_to_now)
                        new_ranges[property] = (new_range_start, current_range_end)

                        if outcome == "A":
                            processed.append(new_ranges)
                        elif outcome != "R":
                            to_process.append((new_ranges, outcome))
                    
                    if current_range_start <= value:
                        new_range_end = min(current_range_end, value)
                        range_up_to_now[property] = (current_range_start, new_range_end)
                    else:
                        break
                case Rule(outcome="A"):
                    processed.append(range_up_to_now)
                
                case Rule(outcome="R"):
                    break
                
                case Rule(outcome=outcome):
                    to_process.append((range_up_to_now, outcome))
    return processed, to_process


In [ ]:
processed, to_process = process_ranges(starting_ranges, workflows)
processed

In [ ]:
to_process

In [ ]:
processed

In [ ]:
from  functools import reduce
from operator import mul

sum([reduce(mul, [y-x+1 for (x,y) in p.values()]) for p in processed])